In [22]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from bs4 import BeautifulSoup
import lxml.html as lh
from datetime import datetime
from lxml import etree

options = webdriver.ChromeOptions() 
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36'})
print(driver.execute_script("return navigator.userAgent;"))
driver.get('https://www.httpbin.org/headers')

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36


In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)     # Display all rows
np.set_printoptions(threshold=np.inf)       # Display entire numpy arrays

### For one webpage

In [3]:
url = f'https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats'
driver.get(url)
driver.implicitly_wait(10)

html_content = driver.page_source

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all table rows in the tbody section
table = soup.find('table', id= f"stats_standard")
# then we can iterate through each row and extract either header or row values:
header = []
rows = []
for j, row in enumerate(table.find_all('tr')):
    if j < 2:
        header = [el.text.strip() for el in row.find_all('th')]
    else:
        rows.append([el.text.strip() for el in row.find_all('td')])
header.pop(0)

data = pd.DataFrame(data=rows, columns=header)
data.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21,2000,36,28,"2,372",26.4,1,3,4,1,0,0,2,0,3.9,3.9,4.2,8.1,43,86,151,0.04,0.11,0.15,0.04,0.15,0.15,0.16,0.31,0.15,0.31,Matches
1,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18,2003,7,0,173,1.9,0,0,0,0,0,0,1,0,0.2,0.2,0.0,0.2,8,6,15,0.00,0.00,0.00,0.00,0.00,0.09,0.03,0.11,0.09,0.11,Matches
2,James Abankwah,ie IRL,DF,Udinese,it Serie A,18,2004,2,1,63,0.7,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
3,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16,2005,1,0,1,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
4,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34,1987,37,37,"3,330",37.0,1,2,3,1,0,0,6,0,2.4,2.4,1.0,3.4,40,215,10,0.03,0.05,0.08,0.03,0.08,0.06,0.03,0.09,0.06,0.09,Matches


### For every webpage

In [24]:
webpage_names = {'stats':'standard', 'keepers':'keeper', 'keepersadv':'keeper_adv', 'shooting':'shooting', 'passing':'passing', 'passing_types':'passing_types', 'gca':'gca', 'defense':'defense', 'possession':'possession',
                'playingtime':'playing_time', 'misc': 'misc'}

In [5]:
for i, j in webpage_names.items():
    url = f'https://fbref.com/en/comps/Big5/{i}/players/Big-5-European-Leagues-Stats'
    driver.get(url)
    driver.implicitly_wait(10)

    html_content = driver.page_source

    # Create a BeautifulSoup object from the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    table = soup.find('table', id= f"stats_{j}")
    # then we can iterate through each row and extract either header or row values:
    header = []
    rows = []
    for j, row in enumerate(table.find_all('tr')):
        if j < 2:
            header = [el.text.strip() for el in row.find_all('th')]
        else:
            rows.append([el.text.strip() for el in row.find_all('td')])
    header.pop(0)
#     # Create the dataframe dynamically
    exec(f"data_{i} = pd.DataFrame(data=rows, columns=header)")  # Replace pd.DataFrame() with your actual data assignment

    exec(f"data_{i}.drop(['Matches'], axis=1, inplace = True)")
    
    exec(f"data_{i}.dropna(how='all', inplace=True)")
#     # Save the dataframe as a CSV file
    exec(f"data_{i}.to_csv('Big5/Big5Player{i}Stats22-23.csv', index=False)")

### Bundesliga 2023-24

In [30]:
for i, j in webpage_names.items():
    url = f'https://fbref.com/en/comps/20/{i}/Bundesliga-Stats'
    driver.get(url)
    driver.implicitly_wait(10)

    html_content = driver.page_source

    # Create a BeautifulSoup object from the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    table = soup.find('table', id= f"stats_{j}")
    # then we can iterate through each row and extract either header or row values:
    header = []
    rows = []
    for j, row in enumerate(table.find_all('tr')):
        if j < 2:
            header = [el.text.strip() for el in row.find_all('th')]
        else:
            rows.append([el.text.strip() for el in row.find_all('td')])
    header.pop(0)
#     # Create the dataframe dynamically
    exec(f"data_{i} = pd.DataFrame(data=rows, columns=header)")  # Replace pd.DataFrame() with your actual data assignment

    exec(f"data_{i}.drop(['Matches'], axis=1, inplace = True)")
    
    exec(f"data_{i}.dropna(how='all', inplace=True)")
#     # Save the dataframe as a CSV file
    exec(f"data_{i}.to_csv('Bundesliga2023-24/BundesligaPlayer{i}Stats23-24.csv', index=False)")

In [31]:
driver.close()